# User study experiments

In [1]:
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
# base_folder = "/home/tum/data/"
# base_folder = "/home/arthur/local_data/" 

traj_n = 40
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = np.concatenate([feature_indices,obj_sim_indices, obj_poses_indices])

/home/arthur/.conda/envs/py38_cu11_2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda


In [2]:
dataset_name = "4D_100k_scaping_factor"

X,Y, data = mr.load_dataset(dataset_name, filter_data = True, base_path=base_folder+"data/")
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

loading dataset:  4D_100k_scaping_factor ...done
raw X: (100000, 953) 	Y: (100000, 160)
filtered X: (99092, 953) 	Y: (99092, 160)


2022-09-09 17:34:05.245145: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-09 17:34:05.246143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-09 17:34:05.246557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-09 17:34:05.247091: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Train X: (69363, 953) 	Y: (69363, 160)
Test  X: (19819, 953) 	Y: (19819, 160)
Val   X: (9910, 953) 	Y: (9910, 160)


In [3]:
%matplotlib qt

%load_ext autoreload
%autoreload 2



In [14]:
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from matplotlib.widgets import Button, TextBox, RadioButtons

import re
from src.functions import *
from matplotlib import rc
import textwrap
import random
import json
from src.config import *
import datetime

w = textwrap.TextWrapper(width=60,break_long_words=False,replace_whitespace=False)

# rc('text', usetex=True)

class User_study_interface():

    def __init__(self, data_ditributions, samples_per_data=1, dis_names=None):
        """data_ditributions: list, with the different data distributions
        samples_per_data: int number of samples per distribution to present the user
        dis_names: list of with the distributuions names"""

        self.fig = plt.figure(figsize=(8,13))
        self.fig.add_subplot(1,1,1,projection='3d')
        self.ax = plt.gca(projection='3d')
        # self.ax.set_xlabel('X axis')
        # self.ax.set_ylabel('Y axis')
        # self.ax.set_zlabel('Z axis')
        self.ax.axes.xaxis.set_ticklabels([])
        self.ax.axes.yaxis.set_ticklabels([])
        self.ax.axes.zaxis.set_ticklabels([])

        self.dis_names = [str(i) for i in range(len(data_ditributions))] if dis_names is None else dis_names
        self.num_samples = samples_per_data*len(data_ditributions) # per ditribution
        exp_data = []
        exp_data_indices = []

        for i,data_dis in enumerate(data_ditributions):
            exp_data = exp_data + random.choices(data_dis, k=samples_per_data)
            exp_data_indices = exp_data_indices + [i]*samples_per_data

        index_shuf = list(range(len(exp_data)))
        random.shuffle(index_shuf)
        print(index_shuf)
        self.exp_data = [exp_data[i] for i in index_shuf]
        self.exp_data_indices = [exp_data_indices[i] for i in index_shuf]


        self.ind =17
        self.ani=None
        self.lines = []
        self.tip_marker = []
        self.objs_scatter = []
        self.objs_text = []
        self.colors = []
        self.enable_save = False
        self.user_answers = {}
        self.user_data = {"name":"", "age":""}
        self.eval_text_list = ['yes, perfect', 'yes, a bit better' , 'same', 'No, a bit wrong', 'No, totally wrong']

        self.sample_i =1

        self.alert_text = None
        self.count_text = self.ax.text2D(0.05, 0.95, str(self.sample_i)+"/"+str(self.num_samples), transform=self.ax.transAxes)

        self.plot_sample(self.exp_data[self.sample_i-1])
        self.setup_bts()
        self.plot_markers()

        plt.show()

    def update(self, num, trajs, lines, tips):

        for i, line in enumerate(lines):
            line.set_data(trajs[i][:2,:num])
            line.set_3d_properties(trajs[i][2,:num])
        for i, tip in enumerate(tips):
            n = max(num-1, 0)
            tip.set_data(trajs[i][:2,n:n+1])
            tip.set_3d_properties(trajs[i][2,n:n+1])
            
    def next_cb(self, event):
        if self.sample_i in self.user_answers.keys():
            self.sample_i += 1
            if self.sample_i > self.num_samples:
                self.reset()
                self.plot_final_screen()
            
            else:
                self.plot_sample(self.exp_data[self.sample_i-1])
                self.reset_bts()
            
                if self.sample_i in self.user_answers.keys():
                    self.radio.set_active(self.user_answers[self.sample_i])

        else:
            if not self.alert_text is None:
                self.alert_text.remove()
            self.alert_text = self.ax.text2D(0.50, -0.1,"Please answer the question first!", transform=self.ax.transAxes, color="red")
        

    def prev_cb(self, event):
        if self.sample_i > 1:
            self.sample_i -= 1
            self.plot_sample(self.exp_data[self.sample_i-1])
            # self.radio.set_active()
            
            self.reset_bts()
            
            if self.sample_i in self.user_answers.keys():
                self.radio.set_active(self.user_answers[self.sample_i])

        # print(self.radio.active)

    def eval_cb(self, labels):
        self.user_answers[self.sample_i] = self.eval_text_list.index(labels)

        # self.radio.remove()
    

    def plot_final_screen(self):
        self.final_bg = plt.axes([0.05, 0.05, 0.9, 0.9], facecolor="white")


        self.final_bg.set_xticks([])
        self.final_bg.set_yticks([])
        final_text = self.final_bg.text(0.05, 0.7,"Thank you for helping us :)", transform=self.final_bg.transAxes, color="Black", fontweight="bold", fontsize=27)

        self.save_text = self.final_bg.text(0.2, 0.63," ", transform=self.final_bg.transAxes, color="white")

        self.ax_name_box = plt.axes([0.2, 0.5, 0.4, 0.04])
        self.name_box = TextBox(self.ax_name_box, 'your name:', initial=self.user_data["name"])
        self.name_box.on_submit(lambda val: self.log_entry(val,"name"))

        self.ax_age_box = plt.axes([0.2, 0.4, 0.4, 0.04])
        self.age_box = TextBox(self.ax_age_box, 'your age:', initial=self.user_data["age"])
        self.age_box.on_submit(lambda val: self.log_entry(val,"age"))

        self.axsave = plt.axes([0.2, 0.2, 0.20, 0.075])
        self.btsave = Button(self.axsave, 'SAVE and EXIT')
        self.btsave.on_clicked(self.save_data)

    def log_entry(self, val, k):
        self.user_data[k] = str(val)

        self.enable_save = True
        for key in self.user_data.keys():
            if self.user_data[key] == "":
                self.enable_save = False

    def save_data(self, name):
        if self.enable_save:
            if not os.path.exists(user_study_folder):
                print("User study folder not found:", user_study_folder)

            answers_per_distribution = {d_name:{a:0 for a in self.eval_text_list} for d_name in self.dis_names}
            for k,v in self.user_answers.items():
                print("V",v, "  K: ",k)
                dis = self.dis_names[self.exp_data_indices[int(k-1)]]
                answer = self.eval_text_list[v]
                answers_per_distribution[dis][answer] = answers_per_distribution[dis][answer] + 1
            print(answers_per_distribution)

            # data_to_save = {"answers_per_distribution": answers_per_distribution, "user_answers": self.user_answer, }
            with open( os.path.join(user_study_folder, self.user_data["name"]+"_"+self.user_data["age"] +"_"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S") +'.json'), 'w') as f:
                json.dump(answers_per_distribution, f)
            plt.close()
        else: 
            if not self.save_text is None:
                self.save_text.remove()
            self.save_text = self.final_bg.text(0.2, 0.63,"Please complete all the fields first", transform=self.final_bg.transAxes, color="red")

    def setup_bts(self):
        
        self.alert_text = self.ax.text2D(0.70, 0.05, "alert", transform=self.ax.transAxes, color="white")
        axcolor = 'lightgoldenrodyellow'

        self.rax = plt.axes([0.15, 0.05, 0.2, 0.15],
                    facecolor=axcolor)
        self.rax.set_title("Does the modified trajectory \n follows the intruction?")

        self.axnext = plt.axes([0.66, 0.05, 0.15, 0.075])
        if self.sample_i < self.num_samples:
            self.bnext = Button(self.axnext, 'Next')
        elif self.sample_i == self.num_samples:
            self.bnext = Button(self.axnext, 'FINISH!')
        

        self.bnext.on_clicked(self.next_cb)

        self.axprev = plt.axes([0.5, 0.05, 0.15, 0.075])
        if self.sample_i > 1:
            self.bprev = Button(self.axprev, 'Previous')

        else:
            self.bprev = Button(self.axprev, 'Previous',color="white")

        self.bprev.on_clicked(self.prev_cb)

        # adjust radio buttons
        self.radio = RadioButtons(self.rax, self.eval_text_list,
                            active=[True,False,False,False],
                            activecolor='r')

        self.radio.on_clicked(self.eval_cb)

    def reset_bts(self):
        self.axprev.remove()
        self.axnext.remove()
        self.rax.remove()
        self.setup_bts()
        
    def plot_sample(self, d):
        

        self.reset()
        
        pts = np.asarray(d["input_traj"])
        pts_new = np.asarray(d["output_traj"])
        text = d["text"]
        obj_names = np.asarray(d["obj_names"])
        obj_pt = np.asarray(d["obj_poses"])
        change_type = d["change_type"]
        image_paths = d["image_paths"]

        objs  = {}
        for x,y,z,name in zip(obj_pt[:,0],obj_pt[:,1],obj_pt[:,2],obj_names):
            objs[name] = {"value":{"obj_p":[x,y,z]}}

        new_pts_list = [pts_new]
        if d["output_traj"] is None:
            new_pts_list = []

        N = 100
        dt = 0.02
        traj_original = incorporate_speed(pts,dt=dt, N=N)
        traj_new = incorporate_speed(pts_new,dt=dt, N=N)
        # fig = plot_samples(text,pts,new_pts_list, objs=objs,fig=figure, show= False, plot_speed=False, labels=["modified"])
    
        self.trajs = [traj_original.T, traj_new.T] 
        self.plot_objs(objs)
        self.plot_trajs(traj_original, traj_new)
        self.ax.set_title("INSTRUCTION:\n\n"+  '\n'.join(w.wrap(text))  , fontsize=18,  fontname="Times New Roman")

        set_axes_equal(self.ax)

        self.animate(N=100)

    def plot_objs(self, objs):
        for i,(name,v) in enumerate(objs.items()):
            x,y,z = v["value"]["obj_p"]
            
            color = self.colors[i] if i < len(self.colors)-1 else "#"+''.join([random.choice('0123456789AB') for j in range(6)])    

            sc = self.ax.scatter(x,y,z, color=color, s=50)
            # print(dir(sc))
            self.objs_scatter.append(sc)
            t = self.ax.text(x, y, z, name, 'x', color=color, ha='center', fontweight="bold")
            self.objs_text.append(t)

    def plot_trajs(self, pts, pts_new):

        alpha = 0.1
        color_original = "red"
        color_modified = "blue"

        x_init, y_init, z_init = pts[:,0],pts[:,1],pts[:,2]
        x_new, y_new, z_new= pts_new[:,0],pts_new[:,1],pts_new[:,2]
        

        line3, = self.ax.plot(x_init, y_init, z_init,alpha=0.9,color=color_original, label="Original")
        self.lines.append(line3)
        line4, = self.ax.plot(x_new, y_new, z_new,alpha=0.9, color=color_modified, label="Modified")
        self.lines.append(line4)
        handles, labels = self.ax.get_legend_handles_labels()
        self.ax.legend(handles[::-1], labels[::-1])

        line1, = self.ax.plot(x_init, y_init, z_init,alpha=alpha,color=color_original, label="Original")
        self.lines.append(line1)
        line2, = self.ax.plot(x_new, y_new, z_new,alpha=alpha, color=color_modified, label="Modified")
        self.lines.append(line2)

        tip_marker1, = self.ax.plot(x_init[0:1], y_init[0:1], z_init[0:1], lw=2, c=color_original, marker='o')
        self.tip_marker.append(tip_marker1)
        tip_marker2, = self.ax.plot(x_new[0:1], y_new[0:1], z_new[0:1], lw=2, c=color_modified, marker='o')
        self.tip_marker.append(tip_marker2)

    def plot_markers(self):

        fs=30
        dist = 0.5
        alpha = 0.2
        color = 'grey'
        self.ax.text(0,dist, 0, "front", 'x',color=color, alpha=alpha, fontsize=fs, ha='center', va='center')
        self.ax.text(-dist,0, 0, "left", 'y',color=color, alpha=alpha, fontsize=fs, ha='center', va='center')
        self.ax.text(0,0, -dist, "bottom", 'x',color=color, alpha=alpha, fontsize=fs, ha='center', va='center')

        # self.ax.text(0,-dist, 0, "back", 'x',color='red', alpha=alpha, fontsize=fs, ha='center', va='center')
        # self.ax.text(dist,0, 0, "right", 'y',color='red', alpha=alpha, fontsize=fs, ha='center', va='center')
        # self.ax.text(0,0, dist, "up", 'x',color='red', alpha=alpha, fontsize=fs, ha='center', va='center')


    def reset(self):
        self.count_text.remove()
        self.count_text = self.ax.text2D(0.05, 0.95, str(self.sample_i)+"/"+str(self.num_samples), transform=self.ax.transAxes)
        
        if not self.alert_text is None:
            self.alert_text.remove()

        for i in range(len(self.lines)):
            self.lines[i].remove()
        for i in range(len(self.tip_marker)):
            self.tip_marker[i].remove()
        for i in range(len(self.objs_text)):
            # print(self.objs_scatter[i])
            # print("\n",self.objs_scatter[i])
            self.objs_scatter[i].remove()
            self.objs_text[i].remove()
        self.objs_scatter = []
        self.objs_text = []
        self.lines = []
        self.tip_marker = []



    def animate(self, N=100):
        
        self.ani = animation.FuncAnimation(self.fig, self.update, N, fargs=(self.trajs,self.lines[:2], self.tip_marker[:2]), interval=1000/N,cache_frame_data=False, blit=False)

user_study = User_study_interface([data[:100],data[100:200]], dis_names=["Ground True", "Generated"])
plt.show()




[0, 1]


In [124]:
plt.close()